   SpaCy Test
==================
Following tutorial from: https://www.analyticsvidhya.com/blog/2020/06/nlp-project-information-extraction/

In [1]:
# << =========================================================================================== >> 
# <<                                           IMPORTS                                           >> 
# << =========================================================================================== >> 

import spacy
from spacy import displacy
from spacy.matcher import Matcher
from nltk.tokenize import sent_tokenize
import visualise_spacy_tree
import pandas as pd
from glob import glob
from pathlib import Path
import re
from typing import List

#### Note:
if you get an error while calling `import_data()` try running the following command:
```py
import nltk
nltk.download('punkt')
```

In [2]:
langmod = spacy.load('en_core_web_sm',disable=['ner','textcat'])

In [3]:
# << =========================================================================================== >> 
# <<                                           GLOBALS                                           >> 
# << =========================================================================================== >> 

COL_LABELS = {'id','content','sents','DLRAMNT'}

  Data Handling
-----------------

In [4]:
INPUT_FILES = glob("./development-docs/*")
ANS_FILES = glob("./development-anskeys/*.key")

### Data Cleaning Operations

In [5]:
def clean_text(content:str) -> str:
    """Clean up the sentences & get rid of unessisary whitespace"""
    # normalize line spacing
    content = re.sub("\r?\n", ' ', content)
    # remove all extra whitespace
    content = re.sub(r"\s+", ' ', content)
    content = re.sub(r"^\s+", '', content)
    # remove the "Reuter"'s tag at the end of the sentence
    content = re.sub(r"\s*[Rr][Ee][Uu][Tt][Ee][Rr][Ss]?\s*$", '', content)
    # tokenize sentences sentences & split [return]
    return content

In [6]:
def import_data(files) -> pd.DataFrame:
    """Produce the dataframes to work with"""
    df = pd.DataFrame(columns=COL_LABELS)
    for file_p,i in zip(files,range(len(files))):
        df.loc[i,'id'] = Path(file_p).stem
        with open(file_p, 'r') as file:
            cont = clean_text(file.read())
            df.loc[i,'content'] = cont
            df.loc[i,'sents'] = sent_tokenize(cont, language="english")
    return df

### Get the data

In [7]:
docs = import_data(INPUT_FILES)
docs

DLRAMNT     id                                            content  \
0       NaN  10042   Meridian Energy Inc and Castone Development C...   
1       NaN  10066   Security Pacific Corp and Users Inc, a credit...   
2       NaN  10158   Americanture Inc said it has purchased Americ...   
3       NaN  10171   Viacom International Inc said it set April 6 ...   
4       NaN   1029   Scott's Hospitality Inc said it acquired all ...   
..      ...    ...                                                ...   
393     NaN   9733   British Petroleum Co Plc said it intended to ...   
394     NaN   9809   Mickelberry Corp said it has completed the pr...   
395     NaN    982   H.J. Heinz chairman Tony O'Reilly would be in...   
396     NaN   9858   MCO Holdings Inc said its shareholders and th...   
397     NaN   9893   Cross and Trecker said it agreed to acquire t...   

                                                 sents  
0    [ Meridian Energy Inc and Castone Development ...  
1    [ Security Pacific Corp and Users Inc, a credi...  
2    [ Americanture Inc said it has purchased Ameri...  
3    [ Viacom International Inc said it set April 6...  
4    [ Scott's Hospitality Inc said it acquired all...  
..                                                 ...  
393  [ British Petroleum Co Plc said it intended to...  
394  [ Mickelberry Corp said it has completed the p...  
395  [ H.J., Heinz chairman Tony O'Reilly would be ...  
396  [ MCO Holdings Inc said its shareholders and t...  
397  [ Cross and Trecker said it agreed to acquire ...  

[398 rows x 4 columns]

## Sale Price Amount Experimental Extraction (SpaCy)

### Manually created SpaCy Pattern

In [24]:
PATS_SALE_PRICE = [
    [ # ...
        {'IS_DIGIT': True, 'LIKE_NUM': True, 'OP':'+'}, # numerical values
        {'LIKE_NUM': True, 'OP':'?', 'IS_ALPHA': True}, #{'IN': ['mln', 'bln']}}, # numerical values like million
        {'POS':'ADJ', 'OP':'?'}, # curency regionally identifiers
        {'POS': 'NOUN'}, # {'IN': ['dlrs', 'p', 'pence', 'pound', 'pounds', 'stg', 'yen']}}, # curency type identifiers
        {'LOWER': "cash", 'OP':'?'}
    ],
]

In [25]:
# test the basics
def dlrs_test(text):
    res = list()
    doc_nlp = langmod(text)
    matcher = Matcher(langmod.vocab)
    matcher.add('DLRAMT', PATS_SALE_PRICE)
    matches = matcher(doc_nlp)
    # finding patterns in the text
    for i in range(0,len(matches)):
        # match: id, start, end
        token = doc_nlp[matches[i][1]:matches[i][2]]
        # append token to list
        res.append(str(token))
    return res if len(res) > 0 else "---"

docs['DLRAMNT'] = docs['content'].apply(dlrs_test)
docs

DLRAMNT     id  \
0                                                  ---  10042   
1                                                  ---  10066   
2                          [11 reorganization, 83 mln]  10158   
3                                                  ---  10171   
4                                 [1986 sales, 20 mln]   1029   
..                                                 ...    ...   
393  [45 pct, 70 dlrs, 70 dlr, 608 mln, 59 pct, 20 ...   9733   
394                                   [51 pct, 49 pct]   9809   
395                                  [80 mln, 800 mln]    982   
396                                                ---   9858   
397                               [1986 sales, 35 mln]   9893   

                                               content  \
0     Meridian Energy Inc and Castone Development C...   
1     Security Pacific Corp and Users Inc, a credit...   
2     Americanture Inc said it has purchased Americ...   
3     Viacom International Inc said it set April 6 ...   
4     Scott's Hospitality Inc said it acquired all ...   
..                                                 ...   
393   British Petroleum Co Plc said it intended to ...   
394   Mickelberry Corp said it has completed the pr...   
395   H.J. Heinz chairman Tony O'Reilly would be in...   
396   MCO Holdings Inc said its shareholders and th...   
397   Cross and Trecker said it agreed to acquire t...   

                                                 sents  \
0    [ Meridian Energy Inc and Castone Development ...   
1    [ Security Pacific Corp and Users Inc, a credi...   
2    [ Americanture Inc said it has purchased Ameri...   
3    [ Viacom International Inc said it set April 6...   
4    [ Scott's Hospitality Inc said it acquired all...   
..                                                 ...   
393  [ British Petroleum Co Plc said it intended to...   
394  [ Mickelberry Corp said it has completed the p...   
395  [ H.J., Heinz chairman Tony O'Reilly would be ...   
396  [ MCO Holdings Inc said its shareholders and t...   
397  [ Cross and Trecker said it agreed to acquire ...   

                                                DLRAMT  
0                                                  ---  
1                                                  ---  
2                          [11 reorganization, 83 mln]  
3                                                  ---  
4                                 [1986 sales, 20 mln]  
..                                                 ...  
393  [45 pct, 70 dlrs, 70 dlr, 608 mln, 59 pct, 20 ...  
394                                   [51 pct, 49 pct]  
395                                  [80 mln, 800 mln]  
396                                                ---  
397                               [1986 sales, 35 mln]  

[398 rows x 5 columns]

In [20]:
docs.loc[393,"DLRAMT"]

['45 pct',
 '70 dlrs',
 '70 dlr',
 '608 mln',
 '59 pct',
 '20 pct',
 '12 dlrs',
 '15 dlrs',
 '1987 profits']

In [17]:
displacy.render(langmod(docs.loc[393,'sents'][0]), style='dep',jupyter=True)